# Introduction to geopandas for analyzing spatial data

* NOTE: this is a slight 2024 update to material originally from https://github.com/uwescience/dssg-pandas-sql-tutorial

[*Geopandas*](https://github.com/geopandas/geopandas) is an excellent extension to the [*Pandas*](https://github.com/pandas-dev/pandas) package for working with tabular data that is georeferenced - for example, Points, Lines, and Polygons that have associated attributes. Geopandas builds upon other great libraries including [*Shapely*](https://github.com/Toblerity/Shapely) and [*Fiona*](https://github.com/Toblerity/Fiona).

There are a lot of tutorials out there: Check out this lesson from a [eScience DSSG tutorial](https://uwescience.github.io/SQL-geospatial-tutorial/) or the [eScience Geohackweek tutorial](https://geohackweek.github.io/vector/).

## This notebook covers the following topics:

* dataset exploration
* basic statistical analysis
* simple geometric operations (convex_hull)
* exporting data for GIS software
* converting coordinates and calculating distance and area
* attribute and spatial joins

In [ ]:
# import all the libraries we are going to use
import pandas as pd
import geopandas as gpd
import shapely

# plotting configuration
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['font.size'] = 16

In [ ]:
#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
# First thing to note is that many geospatial python packages are under active development
# and updated regularly. So the version you're using today will likely be updated in the near 
# future!
print('Pandas version: ', pd.__version__) #0.24.2
print('Geopandas version: ', gpd.__version__) #0.5.0

## Dataset

We'll work with 2 CSV files from here
https://github.com/uwescience/dssg-pandas-sql-tutorial/tree/master/data


**Note that the 'places' information has 9 columns with labels in the first row**
!head ./data/Places_Full.csv

name,address,city,lat,lng,place_id,rating,class,type
Trader Joe's,1700 East Madison Street,Seattle,47.6158665,-122.3099133,ChIJx0M1ztNqkFQRtgspEllQxk8,4.5,supermarket,

**distance data has information about transit between two places**
!head ./data/Dist_Out.csv
class,departure_time,distance.value,duration.value,duration_in_traffic.value,end_lat,end_lon,fare.value,mode,origin,pair,place_id,rank,start_lat,start_lon,status
hospital,2018-06-06 12:41:26,1.47545059,32.18333333,,47.7160344,-122.3122296,,walking,530330001003,530330001003-ChIJEerXpl0RkFQRDOjfwBQDzlA,ChIJEerXpl0RkFQRDOjfwBQDzlA,1,47.72281,-122.28945,OK

# Pandas review

In [ ]:
# Let's work with the smaller 'Places_Full.csv' first
# All Pandas methods are accessed via the 'pd' attribute.
# Since the file is well-formatted, it is easily read into memory:
# We can read small datasets directly from a URL
filePath = 'https://github.com/uwescience/dssg-pandas-sql-tutorial/raw/master/data/Places_Full.csv'
df = pd.read_csv(filePath)

In [ ]:
# 'df' stands for 'Data Frame'. It is essentially a spreadsheet:
# HTML formatted output makes for easier reading
df.head()

In [ ]:
# The dataframe has a lot of convenient methods for fast data exploration
# Start with info to confirm that things were read in correctly
df.info()

In [ ]:
# Simple statistics are obtained for numerical columns
df.describe()

In [ ]:
# Columns can be accessed as dictionary items:
print(df['city'].unique())
# Or accessed as attributes for faster typing:
print(df.city.unique())

In [ ]:
# Gotcha: 
# 'class' attribute is reserved for internal use, but it is also a column heading
#print(df.class.unique()) # This causes an error
print(df['class'].unique()) # This is ok

In [ ]:
# let's change the name of class to avoid that error
df.rename(columns={'class':'place_class'}, inplace=True)
df.place_class.unique()

In [ ]:
# Another common issue with tabular data - 
# certain measurements don't always fit into the defined columns 
# or are missing data, and therefore filled with 'not-a-number (nan)'
# For example, some entries don't have a listed city:
print(df.city.unique())

In [ ]:
# Extract data entries without a city
dfNan = df[df.city.isna()]
dfNan.head()

In [ ]:
# Exercise: print top 10 places, sorted by rating

In [ ]:
# Pandas has some convenient built-in plotting methods using matplotlib
# NOTE: semi-colon prevents bins and data objects from being returned
df.rating.hist(bins=20);

In [ ]:
# Pandas has some convenient built-in plotting methods using matplotlib
# NOTE: semi-colon prevents bins and data objects from being returned
df.boxplot('rating');

In [ ]:
# Exercise: plot box-plots showing 'rating' for each 'place_class'


In [ ]:
# You can work with latitude and longitude infrmation in pandas, but it doesn't integrate well
# with other geospatial tools for analysis and visualization, for example:
df.plot.scatter(x='lng', y='lat', c='rating', cmap='viridis');

# From Pandas to Geopandas

In [ ]:
# Commonly, there at latitude, longitude columns for places, but we must tell geopandas 
# explicity what the geometries (Points) and coordinate reference is (EPSG:4326)
# https://geopandas.readthedocs.io/en/latest/projections.html#coordinate-reference-systems

crs = 'EPSG:4326'
gf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs=crs)

In [ ]:
# NOTE: we've added the special 'geometry' columns
gf.head()

### Visualization

* GeoJSON files are rendered on maps on GitHub.org:
https://github.com/uwescience/dssg2018-geopandasSQL-tutorial/blob/master/data/places.geojson

* There are also some Python plotting libraries that work well with Geopandas including [*cartopy*](https://github.com/SciTools/cartopy), and [*ipyleaflet*](https://github.com/jupyter-widgets/ipyleaflet)

In [ ]:
# Geopandas also has some convenient built-in plotting methods
# Note that we no longer have to specify lat and lon
gf.plot(c=gf.rating);

In [ ]:
# Geopands 'explore' is awesome
gf.explore(column='rating')

## Geometric operations

In [ ]:
# We also now have access to lots of quantifiable spatial information. For example, how large
# is the area containing all these points?
#http://geopandas.org/geometric_manipulations.html
point_collection = shapely.geometry.MultiPoint(gf.geometry.tolist())
polygon = point_collection.convex_hull
polygon

In [ ]:
perimeter = polygon.boundary
perimeter

In [ ]:
# You can save these geometries in a separte geodataframe as you go:
gfShape = gpd.GeoDataFrame(geometry=[polygon], crs = 'EPSG:4326')
gfShape

In [ ]:
# Subsetting is very easy in Geopandas. Often we only want points in a certain bounding box
ymin, ymax, xmin, xmax = [47.55, 47.65, -122.4, -122.3]
subset = gf.cx[xmin:xmax, ymin:ymax]
print(f'original entries = {len(gf)}, subsetted entries = {len(subset)}')

In [ ]:
subset.explore()

## Other free geospatial tools

Geographic Information Systems (GIS) are designed for working with geospatial data. If geopandas is lacking, consider using [QGIS](https://qgis.org/en/site/). Also, [Google Earth Pro](https://www.google.com/earth/desktop/) is now free and is a great visualization tool. Finally, [GDAL/OGR](http://www.gdal.org) is a powerful library with command line tools that many other software packages are based upon.

In [ ]:
# If you want to share the geometry / export it to a GIS program for further analysis:

# There are much more powerful ways to explore geospatial data on interactive maps:
# Uploading this file to GitHub will automatically render on a map
gf.to_file('./data/places.geojson', driver='GeoJSON')

# Default is ESRI shapefile, but Geojson or Geopackage, etc should work to
#http://geopandas.org/io.html
#http://www.geopackage.org

#gfShape.to_file('./data/myshape.shp') #ESRI shapefile is default
gfShape.to_file('./data/myshape.gpkg', driver='GPKG')

## Distance, area, etc.
 
We commonly have points in latitude and longitude, but want to know distances on the ground in meters. These conversions are non-trivial and require some knowledge of different coordinate systems and map projections. Be carefule when making these calculations!
https://support.esri.com/en/technical-article/000011356

In [ ]:
# Excercise: Determine the area of gfShape, what are the unites?
gfShape.area

In [ ]:
# Step 1) convert to a local coordinate system in metric units of distance!

# Google Mercator (EPSG:3857 - https://epsg.io/3857) is another popular 
# projection for web maps, but it is "direction preserving", not "area preserving!"

# A good choice for local distances is the Universal Transverse Mercator
# projection. For Seattle it's EPSG:32610
# http://spatialreference.org/ref/epsg/wgs-84-utm-zone-10n/

gfShape.to_crs('epsg:32610', inplace=True)

In [ ]:
# will be in units of km^2
area = gfShape.area.values[0] * 1e-6
print(f'Points are within an area of {area:.1f} km^2')

In [ ]:
# Exercise:
# Calculate the length of the perimeter


## Load a second database of trip distances

In [ ]:
dfD = gpd.pd.read_csv('https://github.com/uwescience/dssg-pandas-sql-tutorial/raw/master/data/Dist_Out.csv')

In [ ]:
dfD.head()

In [ ]:
dfD.info()

In [ ]:
# Look at attributes (columns) of first entry (row)
dfD.iloc[0]

In [ ]:
# 'distance.value in the table is route distance, what about straight-line distance?
# we already have destination (place_id) lon,lat points in the earlier geodataframe 'gf'
row = 0
dest = dfD.place_id.iloc[row]

gf.query('place_id == @dest')

In [ ]:
# So let's use the start_lat, start_lon for the points in this second geodataframe
crs = 'EPSG:4326'
gfD = gpd.GeoDataFrame(dfD, geometry=gpd.points_from_xy(dfD.start_lon, dfD.start_lat), crs=crs)

In [ ]:
# To get distance let's convert once again to UTM
# Or UTM EPSG:32610
gf_merc = gf.to_crs('epsg:32610')
gfD_merc = gfD.to_crs('epsg:32610')

In [ ]:
# Let's do a sanity check for distance for this first pair:
pointDest = gf_merc.query('place_id == @dest').geometry.values[0]
pointDest.coords.xy

In [ ]:
pointOrig = gfD_merc.geometry.iloc[0]
pointOrig.coords.xy

In [ ]:
dist_m = pointDest.distance(pointOrig)
print(f'Distance in meters: {dist_m:.1f}')
dist_mi = dist_m/1609.34
print(f'Distance in miles: {dist_mi:.1f}')

In [ ]:
# measuring the straight line distance in google earth i get (1.87 km, 1.16 miles)
# Does this make sense? if this is a walking trip that the reported distances would be a bit larger!

In [ ]:
# Excercise: save the Start Point, End Point, and Straight Line between
# them to view in Google Earth or QGIS

In [ ]:
# That was kind of complicated for one point, but it's now easy to get all the 
# straight-line distances between origins and a particular destination:
pId = 'ChIJEerXpl0RkFQRDOjfwBQDzlA'
pointDest = gf_merc.query('place_id == @pId').geometry.values[0]
gfDest = gfD_merc.query('place_id == @pId')
gfDest
# NOTE: lots of the same origin-id. Are these duplicates? 

In [ ]:
gfDest.loc[:,'straightDist'] = gfDest.distance(pointDest).values/1609.34
gfDest.loc[:, ['distance.value','straightDist']]

# Joining databases

In [ ]:
# Given these two databases share a common column (place_id), we could do an
# "attribute" join
# NOTE that the spatial information doesn't really matter in this case!
# http://geopandas.org/mergingdata.html

# Maybe we're only interested in 'departure time, distance and duration'
# for a given destination, but want to keep the location and other attributes:

columns = ('place_id','departure_time','distance.value','duration.value')
gfSubset = gfD.loc[:,columns]
gfSubset
gfPlace = gfSubset.merge(gf, on='place_id')
gfPlace

In [ ]:
# Spatial merging uses spatial relationships (“intersects”, “within” or “contains”)
# For example, let's extract points from the database for a particular neighborhood
# See also:
#https://github.com/seattleio
#https://github.com/geopandas/geopandas/blob/master/examples/spatial_joins.ipynb

URL = 'https://raw.githubusercontent.com/seattleio/seattle-boundaries-data/master/data/neighborhoods.geojson'
gfN = gpd.read_file(URL)
gfN.head()

In [ ]:
gfN.plot()

In [ ]:
neighborhood = 'University District'
gfNeighborhood = gfN.query('nhood == @neighborhood')
gfNeighborhood.plot()

In [ ]:
nhood_places = gpd.sjoin(gf, gfNeighborhood, how="inner", predicate='within')
nhood_places

In [ ]:
# NOTE how columns with the same row were automatically dealt with by 
# appending a _left or _right suffix!
print('{} Places in {}\n'.format(len(nhood_places), neighborhood))
print(nhood_places.name_left)

# Excercise:
# Sort by top 50 ratings in university district (or another district) and save results to both csv and json files.

# Lots of directions to take this in!
- Which neighborhood has the longest travel times?
- What about ratio of straight-line distance to traveled distance?
- Export data for a particular neighborhood
- Incorporate another GIS dataset of your choosing!

# Good luck from here!